# Import Required Libraries
Here, we import the necessary libraries for working with PyTorch and torchvision. These libraries provide tools for creating and training deep learning models, loading datasets, and applying transformations to the data.

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader

# Set Device and Batch Size
We set the device to GPU if it is available; otherwise, we use the CPU. The device determines where the computations will be performed. We also define the batch size, which specifies the number of images processed together during each training iteration.

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 8
print(device)

cuda


# Now we start the processes

In [10]:
# selecting the random seed to ensure reproducability of the results
torch.manual_seed(42)

In [11]:
#lets split the dataset into train and test sets while loading with the Pytorch enabled functionalities
from torch.utils.data import random_split
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_size = 40000
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])
#print(train_size, val_size) #they are 40000 and 10000 respectively.

Files already downloaded and verified


<h3>        Load the Data using DataLoaders</h3>
<li type='square'> They are used to load the data in batches during training and validation. The DataLoader class from PyTorch's torch.utils.data module provides this functionality. We create separate data loaders for the training set (train_loader) and validation set (val_loader) with the specified batch size. The shuffle argument is set to True for the training loader to shuffle the data, and False for the validation loader.

In [12]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified


# <h2>Defining the CNN Model</h2>
<h3><li type="1">Using the ResNet Architecture
<li type= "1"> Using the LeNet Architecture</h3>

# <h3>ResNet (pretrained model)

<li>We define the CNN model using the ResNet-18 architecture. The resnet18 function from torchvision.models provides a pre-trained ResNet-18 model. 
<li>We modify the last fully connected layer (model.fc) to have 10 output classes corresponding to the CIFAR-10 dataset. 
<li>The number of input features (num_features) is obtained from the previous fully connected layer. We move the model to the specified device (GPU or CPU) using the to method.

In [13]:
model = torchvision.models.resnet18(pretrained=False)
num_features = model.fc.in_features
print(model.fc.out_features)
model.fc = nn.Linear(num_features, 10)  # Modify the last layer for 10 output classes
print(model.fc.out_features)


1000
10


<h3>We define Loss function and Optimizer</h3>
<li>We define the loss function, which is the criterion that the model will optimize. 
<li>In this case, we use the cross-entropy loss (nn.CrossEntropyLoss), which is commonly used for multi-class classification problems. 
<li>We also define the optimizer (optim.SGD), which implements stochastic gradient descent. 
<ol><li>It will update the model parameters based on the computed gradients during training. 
<li>We set the learning rate (lr) and momentum values for the optimizer.

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Inside the training loop

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

<h3> Train the Model

In [15]:
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 200 == 199:  # Print the running loss every 200 mini-batches
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/200:.4f}')
            running_loss = 0.0

Epoch [1/10], Batch [200/5000], Loss: 2.3887
Epoch [1/10], Batch [400/5000], Loss: 2.2685
Epoch [1/10], Batch [600/5000], Loss: 2.0885
Epoch [1/10], Batch [800/5000], Loss: 1.9139
Epoch [1/10], Batch [1000/5000], Loss: 1.8757
Epoch [1/10], Batch [1200/5000], Loss: 1.8817
Epoch [1/10], Batch [1400/5000], Loss: 1.8212
Epoch [1/10], Batch [1600/5000], Loss: 1.7804
Epoch [1/10], Batch [1800/5000], Loss: 1.7696
Epoch [1/10], Batch [2000/5000], Loss: 1.7306
Epoch [1/10], Batch [2200/5000], Loss: 1.7500
Epoch [1/10], Batch [2400/5000], Loss: 1.7533
Epoch [1/10], Batch [2600/5000], Loss: 1.6472
Epoch [1/10], Batch [2800/5000], Loss: 1.6794
Epoch [1/10], Batch [3000/5000], Loss: 1.6589
Epoch [1/10], Batch [3200/5000], Loss: 1.6449
Epoch [1/10], Batch [3400/5000], Loss: 1.5983
Epoch [1/10], Batch [3600/5000], Loss: 1.6045
Epoch [1/10], Batch [3800/5000], Loss: 1.5941
Epoch [1/10], Batch [4000/5000], Loss: 1.6018
Epoch [1/10], Batch [4200/5000], Loss: 1.5918
Epoch [1/10], Batch [4400/5000], Loss:

In [16]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in val_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the validation dataset: {accuracy:.2f}%')

Accuracy on the validation dataset: 71.08%


In [17]:
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the validation dataset: {accuracy:.2f}%')

Accuracy on the validation dataset: 71.27%


In [21]:
model.fc.out_features

10

# LeNet

- In this part, we define the LeNet model using the nn.Module class, which is the base class for all neural network models in PyTorch. The LeNet model consists of two main parts: the feature extraction layers (self.features) and the classifier layers (self.classifier).

- The feature extraction layers contain convolutional and pooling operations. These layers extract relevant features from the input images. The classifier layers consist of fully connected (linear) layers that perform the final classification based on the extracted features.

<h3> Implementing the Forward Pass</h3>

- The forward method defines the forward pass computation of the model. 

- It specifies how the input x flows through the different layers of the model. </li>

- In this case, the input x is passed through the feature extraction layers (self.features), reshaped using view to flatten the output, and then passed through the classifier layers (self.classifier) to obtain the final output.

In [18]:
from LeNet import LeNet

<h3>Loading the Model

In [19]:
model_LeNet = LeNet(num_classes=10)

# Step 3: Modify the last layer of the model for 10 output classes
model_LeNet.classifier[4] = nn.Linear(84, 10)

In [20]:
device1 = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_LeNet.to(device1)

# Inside the training loop


LeNet(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)

<h3>Optimizer and Loss</h3>

-  The optimizer's gradients are reset (optimizer.zero_grad()), and the model's forward pass is computed (outputs = model(inputs)).
- The loss is calculated by comparing the predicted outputs with the ground truth labels (loss = criterion(outputs, labels)).
- The gradients are computed (loss.backward()) and the optimizer updates the model's parameters based on the gradients (optimizer.step()).
- The running loss is tracked and printed periodically.

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_LeNet.parameters(), lr=0.025, momentum=0.4)

In [24]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model_LeNet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # Print the running loss every 200 mini-batches
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/200:.4f}')
            running_loss = 0.0

print('Training finished.')

Epoch [1/10], Batch [200/5000], Loss: 0.9740
Epoch [1/10], Batch [400/5000], Loss: 0.9782
Epoch [1/10], Batch [600/5000], Loss: 0.9841
Epoch [1/10], Batch [800/5000], Loss: 0.9998
Epoch [1/10], Batch [1000/5000], Loss: 1.0186
Epoch [1/10], Batch [1200/5000], Loss: 0.9693
Epoch [1/10], Batch [1400/5000], Loss: 1.0097
Epoch [1/10], Batch [1600/5000], Loss: 1.0552
Epoch [1/10], Batch [1800/5000], Loss: 0.9848
Epoch [1/10], Batch [2000/5000], Loss: 1.0498
Epoch [1/10], Batch [2200/5000], Loss: 0.9613
Epoch [1/10], Batch [2400/5000], Loss: 1.0364
Epoch [1/10], Batch [2600/5000], Loss: 1.0232
Epoch [1/10], Batch [2800/5000], Loss: 1.0506
Epoch [1/10], Batch [3000/5000], Loss: 1.0544
Epoch [1/10], Batch [3200/5000], Loss: 1.0625
Epoch [1/10], Batch [3400/5000], Loss: 0.9721
Epoch [1/10], Batch [3600/5000], Loss: 1.0840
Epoch [1/10], Batch [3800/5000], Loss: 1.0420
Epoch [1/10], Batch [4000/5000], Loss: 1.0320
Epoch [1/10], Batch [4200/5000], Loss: 0.9931
Epoch [1/10], Batch [4400/5000], Loss:

<h3>Evaluating the trained model:</h3>

- The code sets the model to evaluation mode (model.eval()).
- It loops over the test dataset and performs inference using the trained model on each batch of data.
- The predicted labels are compared with the ground truth labels to calculate the number of correct predictions.
- The accuracy is computed by dividing the number of correct predictions by the total number of samples and multiplying by 100.
- The accuracy on the validation dataset is printed.

In [ ]:
model_LeNet.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in val_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model_LeNet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the validation dataset: {accuracy:.2f}%')

Accuracy on the validation dataset: 58.17%


# Saving the Models (Not Pruned)

In [48]:
torch.save(model.state_dict(), 'models/modelResNet.pth')
torch.save(model_LeNet.state_dict(), 'models/modelLeNet.pth')
